In [71]:
#Installing requied packages and versions

!pip install pydantic==1.10.5
!pip install gradientai --upgrade

In [72]:
import os
import pandas as pd
import json

#Connecting with Project's Gradient Workplace
os.environ['GRADIENT_ACCESS_TOKEN'] = "0JBggvvXEL3jdLsSwqNH8DlAuMl0XguB"
os.environ['GRADIENT_WORKSPACE_ID'] = "84cac885-e8e9-46d1-a2b3-d12fa07e777c_workspace"

In [74]:
from gradientai import Gradient

with Gradient() as gradient:
    #Choosing our Model
    base_model = gradient.get_base_model(base_model_slug="nous-hermes2")

    #Naming it
    new_model_adapter = base_model.create_model_adapter(
        name="sdgclassifier"
    )
    
    #Getting the model id, for later model usage
    print(f"Created model adapter with id {new_model_adapter.id}")
    
    #Testing the model before finetunning
    samplequery = "### Instructions: Bitte geben Sie mir die Nummer-ID der SDG-Labels an, die in diesem Universitätsmodul: Webtechnologien, gemäß dieser Beschreibung genannt werden können: Studierende, die dieses Modul erfolgreich abschließen, haben einen Überblick über die grundlegenden Technologien des World Wide Web (WWW) und deren Anwendung. Sie haben Kenntnisse von der Funktionsweise des WWW als Overlay Netz im Internet sowie der zugrunde liegenden Kommunikationsinfrastrukturen und deren Protokolle. Darüber hinaus können sie die verschiedenen Generationen des WWW und die gegenwärtigen sowie zukünftigen Entwicklungen benennen.  Studierende sind in der Lage Webinhalte semantisch zu strukturieren, zu gestalten und miteinander zu verlinken. Es ist ihnen möglich dynamische Webseiten zu erstellen. Weiterhin lernen sie server-seitige Anwendungen zu programmieren und diese mit dem Front-End über Schnittstellen zu verbinden. Das Modul gliedert sich wie folgt: - Grundlagen - Die Hauptbestandteile des WWW und seine evolutionäre Entwicklung während der letzten Jahrzehnte - Webpages - Die Strukturierung und Gestaltung von Web-Inhalten mit HTML und CSS - Webanwendungen - Ein Überblick über JavaScript, das DOM und AJAX - Architektur und Kommunikation - Die Grundlagen des OSI-Modells mit der Betrachtung des IP-, TCP-, UDP- und HTTP-Protokolls sowie die Erklärung des REST Architekturstils - Server-seitige Anwendungen - Erstellung von Backend-Applikationen mit Node.js - Ausgewählte APIs und Frameworks Diese Inhalte werden in der Vorlesung vermittelt und in Übungsgruppen vertieft. \n\n### Response:"
    completion = new_model_adapter.complete(query=samplequery, max_generated_token_count=100).generated_output
    print(f"Generated (before fine-tune): {completion}")

Created model adapter with id d531c5f6-fc30-4322-8e0e-73a6a157d824_model_adapter
Generated (before fine-tune):  Die SDG-Labels, die in diesem Universitätsmodul: Webtechnologien, gemäß der Beschreibung genannt werden können, sind:
- SDG 9: Industrie, Innovation und Infrastruktur - durch die Erlernung der Webtechnologien und der Entwicklung von Webanwendungen
- SDG 17: Partnerschaften für die Ziele - durch die Zusammenarbeit und Kooperation zwischen Studierenden in der


In [75]:
# Getting the Data and formating it for proper query format
df = pd.read_csv('German_cleaned_combined2.csv', quotechar='"')
df.replace('[^a-zA-Z0-9\s]', '', regex=True, inplace=True)
# Combine selected columns of each row into one text
df['sample'] = df.apply(lambda row: '<s>### Instruction:\nBitte geben Sie mir nur die Nummer-ID der SDG-Labels an, die in diesem Universitaetsmodul ' + row[0] + ', gemaess dieser Beschreibung genannt werden koennen ' + row[2] + '?\n\n### Response:\n' + row[1] + ' </s>', axis=1)
# Convert the DataFrame to JSON
json_data = df['sample'].to_dict()
json_data = [{"inputs": item} for item in json_data.values()]

with open('data2.json', 'w') as f:
    json.dump(json_data, f)


In [76]:
#This is an old cold for older way of validating the results, still used in order to split the data
from sklearn.model_selection import train_test_split

samples = json_data
# Split json_data into training and validation sets
train_data, val_data = train_test_split(json_data, test_size=0.2, random_state=42)

# Print the lengths of the training and validation sets
print(f"Training set length: {len(train_data)}")
print(f"Validation set length: {len(val_data)}")

Training set length: 318
Validation set length: 80


In [81]:
import random

#Spliting the Data into mini-batches to improve connection with the model server
mini_batches = [train_data[i:i+10] for i in range(0, len(train_data), 10)]

# Fine-tuning the model for set number of epochs, using random rows from our Data
num_epochs = 3
count = 0
while count < num_epochs:
    random_mini_batch = random.choice(mini_batches)
    print(f"Fine-tuning the model, iteration {count + 1}")
    new_model_adapter.fine_tune(samples=random_mini_batch)
    count = count + 1

Fine-tuning the model, iteration 1
Fine-tuning the model, iteration 2
Fine-tuning the model, iteration 3


In [82]:
#Test query after fine-tuning

sample_query = "### Instructions: Bitte geben Sie mir nur die Nummer-ID der SDG-Labels an, die in diesem Universitätsmodul: Webtechnologien, gemäß dieser Beschreibung genannt werden können: Studierende, die dieses Modul erfolgreich abschließen, haben einen Überblick über die grundlegenden Technologien des World Wide Web (WWW) und deren Anwendung. Sie haben Kenntnisse von der Funktionsweise des WWW als Overlay Netz im Internet sowie der zugrunde liegenden Kommunikationsinfrastrukturen und deren Protokolle. Darüber hinaus können sie die verschiedenen Generationen des WWW und die gegenwärtigen sowie zukünftigen Entwicklungen benennen.  Studierende sind in der Lage Webinhalte semantisch zu strukturieren, zu gestalten und miteinander zu verlinken. Es ist ihnen möglich dynamische Webseiten zu erstellen. Weiterhin lernen sie server-seitige Anwendungen zu programmieren und diese mit dem Front-End über Schnittstellen zu verbinden. Das Modul gliedert sich wie folgt: - Grundlagen - Die Hauptbestandteile des WWW und seine evolutionäre Entwicklung während der letzten Jahrzehnte - Webpages - Die Strukturierung und Gestaltung von Web-Inhalten mit HTML und CSS - Webanwendungen - Ein Überblick über JavaScript, das DOM und AJAX - Architektur und Kommunikation - Die Grundlagen des OSI-Modells mit der Betrachtung des IP-, TCP-, UDP- und HTTP-Protokolls sowie die Erklärung des REST Architekturstils - Server-seitige Anwendungen - Erstellung von Backend-Applikationen mit Node.js - Ausgewählte APIs und Frameworks Diese Inhalte werden in der Vorlesung vermittelt und in Übungsgruppen vertieft. \n\n### Response:"
completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
print(f"Generated (after fine-tune): {completion}")


Generated (after fine-tune): 4 9 16 


In [83]:
#Testing the model and checking it accuracy
test_count = 0
correct_class = 0

numberOfTests = 10
while test_count < numberOfTests:
    df = pd.read_csv('German_cleaned_combined2.csv', quotechar='"')
    random_row = df.sample(n=1)
    random_row.replace('[^a-zA-Z0-9\s]', '', regex=True, inplace=True)
    expected_results = random_row.iloc[0, 1]
    
    # Combine selected columns of each row into one text
    test_query = '<s>### Instruction:\nBitte geben Sie mir nur die Nummer-ID der SDG-Labels an, die in diesem Universitaetsmodul ' + random_row.iloc[0, 0] + ', gemaess dieser Beschreibung genannt werden koennen ' + random_row.iloc[0, 2] + '?\n\n### Response:'
    completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
    if expected_results.replace(" ", "") ==  completion.replace(" ", ""):
        correct_class = correct_class + 1
    print("Expected: " + expected_results + " Actual:" +completion)
    test_count = test_count + 1
results = correct_class * 100 / numberOfTests
print("Accuracy: " + str(results) + "%")

Expected: 7 9 11 13 Actual:4 9 16 
Expected: 4 9 Actual:4 9 16 
Expected: 4 9 Actual:4 9 16 
Expected: 4 9 16 Actual:4 9 16 
Expected: 12 Actual:4 9 16 
Expected: 4 9 11 Actual:4 9 16 
Expected: 4 9 Actual:4 9 16 
Expected: 7 9 11 13 Actual:4 9 16 
Expected: 4 9 12 Actual:4 9 16 
Expected: 4 9 11 Actual:4 9 16 
Accuracy: 10.0%
